In [8]:
from pathlib import Path
import subprocess
import os
import shutil
import re
import pandas as pd

from tqdm.autonotebook import tqdm

/home1/hannah/miniconda3/envs/xeus-cling/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [9]:
HWP = Path("/home1/hannah/projects/homeworks212/needs_grading/")
roster = pd.read_csv(HWP/"classroom_roster.csv").fillna(value="NULL")
assignments = Path("Homework 1 Hello World-02-21-2019-02-45-18/")

In [10]:
nums = re.compile(r"\d+")

In [12]:
tqdm.pandas()

In [14]:
def grade_student(row):
    try:
        os.chdir(HWP/assignments/row['github_username'])
    except FileNotFoundError:
        grade = 0
        error = "no submission"
        return
    
    if Path("build").exists():
        shutil.rmtree("build")
    os.mkdir("build")
    os.chdir("build")
    
    for args in ([["cmake",".."], ["make"], ["./tests"]]):
        process = subprocess.run(args, capture_output=True, text=True)
        if process.returncode != 0:
            grade = process.stderr
            error = 0
            return
        
    if args != ["./tests"]:
        return
    
    output = process.stdout.split("\n")
    scores = nums.findall(output[-2])
    grade = int(scores[0])/1
    error = "NULL"
    return grade, error

In [21]:
grade_student(roster.iloc[0])

(1.0, 'NULL')

In [ ]:
roster['output'] = roster.progress_apply(grade_student, axis=1)